# CTR Feature EDA

`train.parquet`의 피처 분포를 검토해 전처리 전략을 수립하기 위한 노트북입니다.

## 1. 환경 설정 & 데이터 로딩

필요 라이브러리를 임포트하고, Parquet에서 일정량의 데이터를 샘플링해 분석에 사용합니다.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow.parquet as pq

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)

RAW_DIR = Path('../data/raw')
TRAIN_PATH = RAW_DIR / 'train.parquet'


In [2]:
def load_train_sample(path: Path, n_rows: int = 200_000, batch_size: int = 50_000) -> pd.DataFrame:
    '''Parquet 파일에서 선두 n_rows를 로드한다.'''
    pf = pq.ParquetFile(path)
    pieces = []
    rows = 0
    for batch in pf.iter_batches(batch_size=batch_size):
        pdf = batch.to_pandas()
        pieces.append(pdf)
        rows += len(pdf)
        if rows >= n_rows:
            break
    if not pieces:
        raise RuntimeError(f'No data loaded from {path}')
    df = pd.concat(pieces, ignore_index=True)
    return df.iloc[:n_rows].copy()


In [3]:
SAMPLE_ROWS = 200_000
train_df = load_train_sample(TRAIN_PATH, n_rows=SAMPLE_ROWS)
print(f'train_df shape: {train_df.shape}')
train_df.head()


train_df shape: (200000, 119)


,gender,age_group,inventory_id,day_of_week,hour,seq,l_feat_1,l_feat_2,l_feat_3,l_feat_4,l_feat_5,l_feat_6,l_feat_7,l_feat_8,l_feat_9,l_feat_10,l_feat_11,l_feat_12,l_feat_13,l_feat_14,l_feat_15,l_feat_16,l_feat_17,l_feat_18,l_feat_19,l_feat_20,l_feat_21,l_feat_22,l_feat_23,l_feat_24,l_feat_25,l_feat_26,l_feat_27,feat_e_1,feat_e_2,feat_e_3,feat_e_4,feat_e_5,feat_e_6,feat_e_7,feat_e_8,feat_e_9,feat_e_10,feat_d_1,feat_d_2,feat_d_3,feat_d_4,feat_d_5,feat_d_6,feat_c_1,feat_c_2,feat_c_3,feat_c_4,feat_c_5,feat_c_6,feat_c_7,feat_c_8,feat_b_1,feat_b_2,feat_b_3,feat_b_4,feat_b_5,feat_b_6,feat_a_1,feat_a_2,feat_a_3,feat_a_4,feat_a_5,feat_a_6,feat_a_7,feat_a_8,feat_a_9,feat_a_10,feat_a_11,feat_a_12,feat_a_13,feat_a_14,feat_a_15,feat_a_16,feat_a_17,feat_a_18,history_a_1,history_a_2,history_a_3,history_a_4,history_a_5,history_a_6,history_a_7,history_b_1,history_b_2,history_b_3,history_b_4,history_b_5,history_b_6,history_b_7,history_b_8,history_b_9,history_b_10,history_b_11,history_b_12,history_b_13,history_b_14,history_b_15,history_b_16,history_b_17,history_b_18,history_b_19,history_b_20,history_b_21,history_b_22,history_b_23,history_b_24,history_b_25,history_b_26,history_b_27,history_b_28,history_b_29,history_b_30,clicked
0,1.0,7.0,36,5,13,"9,18,269,516,57,97,527,74,317,311,269,479,57,74,317,77,58,58,540,269,300,479,57,58,58,74,77,318,479,35,57,516,97,479...",1.0,2.0,1.0,23.0,1.0,1.0,193.0,2.0,50.0,118.0,743.0,2877.0,2.0,1591.0,1058.0,2.0,50.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1129.0,5.0,2.0,65.000000,-4230.666504,23.863636,-0.05,0.019608,-0.012987,0.6250,-242.488892,-0.612903,-436.228577,0.38961,2.0,1.764706,1.444444,-0.303030,-0.422535,58.840435,0.043634,66.696426,28.672052,45.806812,7.606154,73.891685,26.482759,4.845361,-4.337349,47.611111,0.907216,-75.421051,1961.083374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4727,0.0,0.0,0.0,0.0,0.032765,-0.055556,0.024390,-326.857147,-0.014493,-183.285721,-13.596154,0.115821,0.138626,0.047507,0.050622,0.026479,0.001558,0.024922,0.051401,0.004673,0.021028,0.072428,0.007009,0.028816,0.058410,0.000779,0.072428,0.016355,0.011682,0.010124,0.002336,0.008567,0.070092,0.070092,0.011682,0.004673,0.087226,0.049843,0.015576,0.040498,0.051401,0
1,1.0,7.0,2,5,08,"9,144,269,57,516,97,527,74,315,317,311,269,479,57,74,315,317,58,58,540,269,479,300,57,58,58,74,317,75,269,450,15,269...",2.0,2.0,3.0,17.0,193.0,116.0,164.0,2.0,14.0,109.0,674.0,218.0,2.0,122.0,751.0,1.0,14.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1129.0,16.0,1.0,65.000000,-1346.648193,4.545455,-0.05,0.019608,-0.012987,104.6875,-194.066666,-52.709679,-390.728577,0.38961,2.0,1.764706,12.865979,-0.303030,-0.422535,58.840435,0.062986,23.371054,28.672052,31.121622,7.223301,73.891685,24.990477,4.432990,-4.469880,14.648149,0.927835,-34.771931,1195.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.7767,0.0,0.0,0.0,0.0,0.009340,0.000000,0.000000,-382.285706,0.000000,-176.000000,-11.442307,0.068794,0.072179,0.049471,0.052715,0.027574,0.001622,0.025952,0.053526,0.004866,0.021897,0.075423,0.007299,0.030007,0.060825,0.000811,0.075423,0.017031,0.012165,0.010543,0.002433,0.008921,0.072990,0.072990,0.012165,0.004866,0.045416,0.051904,0.016220,0.042172,0.026763,0
2,1.0,7.0,36,5,11,"269,516,57,97,165,527,74,77,317,269,75,450,15,269,483,75,450,15,269,483,75,450,207,452,452,51,269,49,508,450,74,77,3...",1.0,2.0,1.0,7.0,675.0,85.0,227.0,2.0,362.0,212.0,1029.0,3916.0,1.0,2924.0,2304.0,2.0,362.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1129.0,16.0,1.0,64.638885,-3195.388916,22.727272,-0.05,0.019608,-0.051948,13.6875,-305.844452,-12.677420,-600.885742,0.38961,2.0,1.764706,15.954545,-0.222222,-0.394366,58.840435,0.033670,43.445312,28.672052,41.673958,7.606154,73.891685,27.473171,4.402062,-0.819277,24.388889,0.917526,-67.701752,2386.083252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6597,0.0,0.0,0.0,0.0,0.078562,-0.111111,0.097561,-409.000000,-0.014493,-224.714279,-13.942307,0.112947,0.169634,0.038753,0.041295,0.021600,0.001271,0.020330,0.041930,0.003812,0.017153,0.059083,0.0

## 2. 기본 정보

데이터 타입, 기초 통계, 타깃 분포 등을 확인합니다.

In [4]:
train_df.info(max_cols=200)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 119 columns):
 #    Column        Non-Null Count   Dtype  
---   ------        --------------   -----  
 0    gender        199516 non-null  object 
 1    age_group     199516 non-null  object 
 2    inventory_id  200000 non-null  object 
 3    day_of_week   200000 non-null  object 
 4    hour          200000 non-null  object 
 5    seq           200000 non-null  object 
 6    l_feat_1      200000 non-null  float32
 7    l_feat_2      199516 non-null  float32
 8    l_feat_3      200000 non-null  float32
 9    l_feat_4      200000 non-null  float32
 10   l_feat_5      200000 non-null  float32
 11   l_feat_6      200000 non-null  float32
 12   l_feat_7      200000 non-null  float32
 13   l_feat_8      199516 non-null  float32
 14   l_feat_9      200000 non-null  float32
 15   l_feat_10     200000 non-null  float32
 16   l_feat_11     200000 non-null  float32
 17   l_feat_12     200000 non-nu

In [5]:
target_col = 'clicked'
sequence_col = 'seq'
possible_id_cols = [col for col in train_df.columns if col.lower() in {'id', 'sample_id', 'row_id'}]
print('Target:', target_col)
print('Sequence column:', sequence_col)
print('Possible ID columns:', possible_id_cols)


Target: clicked
Sequence column: seq
Possible ID columns: []


In [6]:
target_stats = train_df[target_col].value_counts().to_frame('count')
target_stats['ratio'] = target_stats['count'] / len(train_df)
target_stats


,count,ratio
clicked,,
0,196041,0.980205
1,3959,0.019795


In [7]:
train_df.describe(include=[np.number]).T.head(20)


,count,mean,std,min,25%,50%,75%,max
l_feat_1,200000.0,1.848410,0.358624,1.0,2.0,2.0,2.0,2.0
l_feat_2,199516.0,1.836870,0.369485,1.0,2.0,2.0,2.0,2.0
l_feat_3,200000.0,2.358285,0.730137,1.0,2.0,3.0,3.0,3.0
l_feat_4,200000.0,9.983255,6.205117,1.0,7.0,7.0,14.0,26.0
l_feat_5,200000.0,443.598267,321.315765,1.0,109.0,444.0,728.0,1026.0
l_feat_6,200000.0,349.409485,264.545746,1.0,85.0,302.0,557.0,870.0
l_feat_7,200000.0,161.896530,75.502060,1.0,100.0,169.0,227.0,301.0
l_feat_8,199516.0,1.993945,0.077576,1.0,2.0,2.0,2.0,2.0
l_feat_9,200000.0,237.115387,133.410706,1.0,118.0,254.0,362.0,461.0
l_feat_10,200000.0,130.619461,65.146385,1.0,91.0,131.0,187.0,254.0


In [8]:
train_df.describe(include='object').T.head(20)


,count,unique,top,freq
gender,199516,2,1.0,125602
age_group,199516,8,7.0,69488
inventory_id,200000,18,2,49887
day_of_week,200000,2,5,152306
hour,200000,24,08,14515
seq,200000,190657,144,279


## 3. 피처 타입 분류

범주형(sparse), 시퀀스, Dense 피처 후보를 정리합니다.

In [9]:
sparse_features = ['gender', 'age_group', 'inventory_id']

feature_exclude = {target_col, sequence_col, 'ID'}
feature_exclude.update(sparse_features)

dense_features = [col for col in train_df.columns if col.startswith('l_feat_') or col.startswith('history_')]

other_numeric = [col for col in train_df.select_dtypes(include=[np.number]).columns
                 if col not in feature_exclude and col not in dense_features]

print(f'Sparse features ({len(sparse_features)}):', sparse_features)
print(f'Dense features (l_feat/history_ prefix) count: {len(dense_features)}')
print(f'Other numeric candidates: {other_numeric[:10]}')


Sparse features (3): ['gender', 'age_group', 'inventory_id']
Dense features (l_feat/history_ prefix) count: 64
Other numeric candidates: ['feat_e_1', 'feat_e_2', 'feat_e_3', 'feat_e_4', 'feat_e_5', 'feat_e_6', 'feat_e_7', 'feat_e_8', 'feat_e_9', 'feat_e_10']


In [10]:
missing_ratio = train_df.isna().mean().sort_values(ascending=False)
missing_ratio.head(20)


feat_e_3       0.08182
feat_a_8       0.00253
feat_a_18      0.00253
feat_a_17      0.00253
feat_a_16      0.00253
feat_a_15      0.00253
feat_a_14      0.00253
feat_a_12      0.00253
feat_a_11      0.00253
feat_a_10      0.00253
feat_a_13      0.00253
feat_a_7       0.00253
feat_a_6       0.00253
feat_a_5       0.00253
feat_a_4       0.00253
feat_a_3       0.00253
feat_a_2       0.00253
feat_a_1       0.00253
feat_a_9       0.00253
history_a_4    0.00242
dtype: float64

## 4. Sparse Feature 분석

카테고리별 분포와 클릭률을 살펴봅니다.

In [12]:
def categorical_summary(df: pd.DataFrame, col: str, target: str, top_n: int = 20) -> pd.DataFrame:
    vc = df[col].value_counts(dropna=False).head(top_n)
    ctr = df.groupby(col)[target].mean()
    summary = pd.DataFrame({'count': vc, 'ratio': vc / len(df)})
    summary['ctr'] = ctr
    return summary.sort_values('count', ascending=False)

for col in sparse_features:
    print(f'=== {col} ===')
    display(categorical_summary(train_df, col, target_col))


=== gender ===


,count,ratio,ctr
gender,,,
1.0,125602,0.62801,0.019840
2.0,73914,0.36957,0.019793
None,484,0.00242,NaN


=== age_group ===


,count,ratio,ctr
age_group,,,
7.0,69488,0.347440,0.018507
8.0,51360,0.256800,0.024046
6.0,42584,0.212920,0.015992
5.0,10804,0.054020,0.019530
4.0,9865,0.049325,0.019665
3.0,8615,0.043075,0.020662
2.0,6282,0.031410,0.023241
1.0,518,0.002590,0.046332
None,484,0.002420,NaN


=== inventory_id ===


,count,ratio,ctr
inventory_id,,,
2,49887,0.249435,0.009321
36,30318,0.151590,0.014315
37,27031,0.135155,0.032555
29,18945,0.094725,0.021430
42,18146,0.090730,0.021988
46,14048,0.070240,0.019505
88,12829,0.064145,0.038273
43,8224,0.041120,0.014713
31,7733,0.038665,0.022501


## 5. Dense Feature 분포

연속형 피처의 기초 통계와 분포를 파악합니다.

In [13]:
dense_summary = train_df[dense_features].describe().T
dense_summary[['mean', 'std', 'min', 'max']].head(20)


,mean,std,min,max
l_feat_1,1.848410,0.358624,1.0,2.0
l_feat_2,1.836870,0.369485,1.0,2.0
l_feat_3,2.358285,0.730137,1.0,3.0
l_feat_4,9.983255,6.205117,1.0,26.0
l_feat_5,443.598267,321.315765,1.0,1026.0
l_feat_6,349.409485,264.545746,1.0,870.0
l_feat_7,161.896530,75.502060,1.0,301.0
l_feat_8,1.993945,0.077576,1.0,2.0
l_feat_9,237.115387,133.410706,1.0,461.0
l_feat_10,130.619461,65.146385,1.0,254.0


In [14]:
zero_ratio = (train_df[dense_features] == 0).mean().sort_values(ascending=False)
zero_ratio.head(20)


history_a_5     0.530220
history_a_2     0.377070
history_a_3     0.224060
l_feat_18       0.034855
history_a_7     0.001895
history_b_1     0.001895
history_b_25    0.000800
history_b_11    0.000800
history_b_29    0.000800
history_b_3     0.000800
history_b_28    0.000800
history_b_4     0.000800
history_b_27    0.000800
history_b_26    0.000800
history_b_5     0.000800
history_b_6     0.000800
history_b_7     0.000800
history_b_8     0.000800
history_b_9     0.000800
history_b_10    0.000800
dtype: float64

## 6. Dense Feature 간 상관관계(샘플)

상관계수가 큰 조합을 찾아봅니다.

In [15]:
corr_sample = train_df[dense_features].sample(n=20, axis=1, random_state=42)
corr_matrix = corr_sample.corr()
corr_matrix


,history_b_19,history_b_25,l_feat_1,history_b_11,l_feat_6,history_b_3,l_feat_17,l_feat_13,l_feat_26,history_b_28,history_b_23,l_feat_10,history_b_7,l_feat_14,history_b_1,history_b_17,l_feat_4,l_feat_18,history_b_16,l_feat_9
history_b_19,1.000000,0.999610,0.036809,0.999677,-0.017030,0.998249,-0.038122,-0.087861,0.014716,0.999346,0.999730,0.035141,0.999329,0.015769,0.751574,0.996434,-0.023518,0.010109,0.999745,-0.038122
history_b_25,0.999610,1.000000,0.036827,0.999795,-0.017162,0.997932,-0.038099,-0.088094,0.014761,0.999455,0.999850,0.035014,0.999445,0.015621,0.751501,0.996534,-0.023696,0.009993,0.999865,-0.038099
l_feat_1,0.036809,0.036827,1.000000,0.036844,0.245105,0.036385,-0.131040,-0.084470,-0.000367,0.036874,0.036812,0.059807,0.036851,-0.080657,0.045344,0.036876,-0.021659,0.009374,0.036826,-0.131040
history_b_11,0.999677,0.999795,0.036844,1.000000,-0.017152,0.998010,-0.038158,-0.087923,0.014731,0.999528,0.999915,0.035023,0.999514,0.015587,0.751534,0.996627,-0.023548,0.009932,0.999930,-0.038158
l_feat_6,-0.017030,-0.017162,0.245105,-0.017152,1.000000,-0.017401,0.280297,0.289555,0.004662,-0.017159,-0.017121,0.318508,-0.016924,0.300296,-0.024250,-0.017539,0.052512,-0.005917,-0.017134,0.280297
history_b_3,0.998249,0.997932,0.036385,0.998010,-0.017401,1.000000,-0.037526,-0.088222,0.014756,0.997832,0.998085,0.035060,0.997652,0.015771,0.753329,0.994891,-0.023783,0.010044,0.998067,-0.037526
l_feat_17,-0.038122,-0.038099,-0.131040,-0.038158,0.280297,-0.037526,1.000000,-0.000400,0.005023,-0.037839,-0.038145,0.325748,-0.038096,0.460536,-0.047100,-0.037280,0.097067,-0.027270,-0.038127,1.000000
l_feat_13,-0.087861,-0.088094,-0.084470,-0.087923,0.289555,-0.088222,-0.000400,1.000000,-0.010214,-0.088085,-0.088025,-0.219331,-0.087932,-0.008676,-0.104323,-0.087800,-0.021676,0.007520,-0.088066,-0.000400
l_feat_26,0.014716,0.014761,-0.000367,0.014731,0.004662,0.014756,0.005023,-0.010214,1.000000,0.014668,0.014712,-0.002967,0.014761,0.001174,0.014052,0.014699,0.013298,-0.016479,0.014718,0.005023
history_b_28,0.999346,0.999455,0.036874,0.999528,-0.017159,0.997832,-0.037839,-0.088085,0.014668,1.000000,0.999574,0.035053,0.999172,0.015706,0.751936,0.996368,-0.023727,0.009995,0.999589,-0.037839


## 7. Sequence Feature 분석

`seq` 길이, 고유 토큰 수 등을 계산합니다.

In [16]:
def sequence_stats(series: pd.Series) -> pd.DataFrame:
    lengths = series.fillna('').map(lambda s: len(s.split(',')) if s else 0)
    summary = {
        'min': lengths.min(),
        'max': lengths.max(),
        'mean': lengths.mean(),
        'median': lengths.median(),
        'std': lengths.std(),
        'p05': lengths.quantile(0.05),
        'p25': lengths.quantile(0.25),
        'p75': lengths.quantile(0.75),
        'p95': lengths.quantile(0.95),
    }
    return pd.DataFrame(summary, index=['seq_length'])

seq_lengths = sequence_stats(train_df[sequence_col])
seq_lengths


,min,max,mean,median,std,p05,p25,p75,p95
seq_length,1,6590,557.61736,464.0,449.466558,33.0,197.0,816.0,1397.0


In [17]:
token_counts = {}
for seq in train_df[sequence_col].dropna().head(100_000):
    for token in seq.split(','):
        token = token.strip()
        if token:
            token_counts[token] = token_counts.get(token, 0) + 1

unique_tokens = len(token_counts)
print('Unique tokens observed (first 100k rows):', unique_tokens)
print('Top 10 tokens:')
top_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)[:10]
for token, freq in top_tokens:
    print(f'{token}: {freq}')


Unique tokens observed (first 100k rows): 517
Top 10 tokens:
74: 4320833
101: 4114249
479: 3892251
57: 3872861
408: 3307577
269: 3112363
77: 2786314
317: 1918908
318: 1608600
207: 1455125


## 8. 클릭률과의 상관 탐색

피처와 타깃 사이 관계 신호를 간단히 확인합니다.

In [18]:
dense_corr_with_target = train_df[dense_features + [target_col]].corr()[target_col].drop(target_col).sort_values(ascending=False)
dense_corr_with_target.head(20)


history_a_3     0.073059
history_a_1     0.066307
history_b_2     0.058077
history_b_30    0.056579
history_b_1     0.054356
history_b_10    0.046726
history_b_17    0.041069
history_b_15    0.040623
history_b_3     0.040490
history_b_9     0.040474
history_b_20    0.040463
history_b_27    0.040178
history_b_5     0.039529
history_b_28    0.039507
history_b_26    0.039472
history_b_14    0.039267
history_b_12    0.039208
history_b_19    0.039090
history_b_8     0.038979
history_b_11    0.038969
Name: clicked, dtype: float64

In [19]:
dense_corr_with_target.tail(20)


l_feat_6       0.004454
l_feat_12      0.003724
l_feat_9       0.003324
l_feat_17      0.003324
l_feat_22      0.002292
l_feat_14      0.001884
l_feat_2       0.001672
l_feat_4       0.000621
l_feat_26     -0.000002
l_feat_25     -0.001434
l_feat_24     -0.001970
l_feat_8      -0.002343
l_feat_18     -0.003123
l_feat_21     -0.012568
l_feat_13     -0.013815
l_feat_16     -0.016630
history_a_5   -0.069302
history_a_2   -0.074520
l_feat_20           NaN
l_feat_23           NaN
Name: clicked, dtype: float64

In [20]:
for col in sparse_features:
    ctr = train_df.groupby(col)[target_col].mean().sort_values(ascending=False)
    print(f'Top CTR categories for {col}:')
    display(ctr.head(5))


Top CTR categories for gender:


gender
1.0    0.019840
2.0    0.019793
Name: clicked, dtype: float64

Top CTR categories for age_group:


age_group
1.0    0.046332
8.0    0.024046
2.0    0.023241
3.0    0.020662
4.0    0.019665
Name: clicked, dtype: float64

Top CTR categories for inventory_id:


inventory_id
92    0.071429
41    0.041149
88    0.038273
37    0.032555
91    0.028803
Name: clicked, dtype: float64

## 9. 전처리 전략 메모

EDA 결과를 기반으로 전처리 가이드를 정리합니다.


- Sparse feature는 범주 수가 크므로 임베딩 기반 인코딩(DIN/DeepCTR) 적용, rare category는 빈도 기준으로 `UNK` 처리 고려.
- Dense feature는 0 비율이 높은 항목을 따로 모니터링하고, 필요 시 log/standard scaling 적용.
- `seq` 길이가 매우 길기 때문에 maxlen을 1,000± 수준으로 설정하고, attention/LSTM 모델에 맞춰 패딩 및 마스크 처리를 진행.
- 결측 비율이 높은 피처는 대체값 또는 드롭 전략을 세우고, `history_` 계열은 최근 구간으로 축약해 정보 손실을 줄일 수 있는지 검토.
